In [ ]:
# Importing necessary libraries
import numpy as np
import librosa
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN, BatchNormalization
from keras.optimizers import Adam

In [ ]:
# Define function to extract features from audio
def extract_features(file_path):
    try:
        audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30) 
        mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=13)
        mfccs_processed = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_path)
        return None 
     
    return mfccs_processed

# Define function to load data and extract features
def load_data(data_path):
    labels = []
    features = []
    
    for folder in os.listdir(data_path):
        genre_folder = os.path.join(data_path, folder)
        for file in os.listdir(genre_folder):
            file_path = os.path.join(genre_folder, file)
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(folder)
                
    return np.array(features), np.array(labels)

In [ ]:
# Load data
data_path = 'Data/genres_original'  # Update with your dataset path
features, labels = load_data(data_path)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(encoded_labels)

In [ ]:
features.shape
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_categorical, test_size=0.2, random_state=42)
X_train.shape[0]
# Reshape input data to have the appropriate shape for RNN
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [ ]:
#experiment_1

# Define a function to create the model
def create_model(units_1, units_2, units_3, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(SimpleRNN(units=units_1, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())

    model.add(SimpleRNN(units=units_2, return_sequences=True))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(SimpleRNN(units=units_3))

    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize variables to store best score and parameters
best_score = 0
best_params = {}
best_model = None

# Define lists of hyperparameters to search over
learning_rates = [0.001, 0.0015, 0.002]
dropout_rates = [0.1, 0.2, 0.3]
batch_sizes = [32, 64, 128, 256]
epochs_list = [30, 40, 50, 60]

# Define lists of units for SimpleRNN layers
units_list_1 = [64, 128, 256]
units_list_2 = [32, 64, 128]
units_list_3 = [16, 32, 64]

# Iterate over all combinations of hyperparameters
for lr in learning_rates:
    for dropout_rate in dropout_rates:
        for batch_size in batch_sizes:
            for epochs in epochs_list:
                for units_1 in units_list_1:
                    for units_2 in units_list_2:
                        for units_3 in units_list_3:
                            # Create model with current hyperparameters
                            model = create_model(learning_rate=lr, dropout_rate=dropout_rate, units_1=units_1, units_2=units_2, units_3=units_3)
                            
                            # Train the model
                            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
                            
                            # Evaluate the model on validation data
                            _, accuracy = model.evaluate(X_test, y_test, verbose=0)
                            
                            # Store the accuracy and parameters
                            params = {'learning_rate': lr, 'dropout_rate': dropout_rate, 'batch_size': batch_size, 'epochs': epochs, 'units_1': units_1, 'units_2': units_2, 'units_3': units_3}
                            print("Accuracy:", accuracy, "Params:", params)
                            
                            # Check if current model has the best score
                            if accuracy > best_score:
                                best_score = accuracy
                                best_params = params
                                best_model = model

# Print the best score and parameters
print("Best Mean Test Accuracy:", best_score)
print("Best Parameters:", best_params)

# Evaluate the best model on the test data
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Generate a classification report for the best model
y_pred = best_model.predict_classes(X_test)
report = classification_report(np.argmax(y_test, axis=1), y_pred)
print("Classification Report:")
print(report)

In [ ]:
#experiment_2
# Define a function to create the model
def create_model(units_1, units_2, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(SimpleRNN(units=units_1, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())

    model.add(SimpleRNN(units=units_2))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())

    model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize variables to store best score and parameters
best_score = 0
best_params = {}
best_model = None

# Define lists of hyperparameters to search over
learning_rates = [0.001, 0.0015, 0.002]
dropout_rates = [0.1, 0.2, 0.3]
batch_sizes = [32, 64, 128, 256]
epochs_list = [30, 40, 50, 60]

# Define lists of units for SimpleRNN layers
units_list_1 = [64, 128, 256]
units_list_2 = [32, 64, 128]

# Iterate over all combinations of hyperparameters
for lr in learning_rates:
    for dropout_rate in dropout_rates:
        for batch_size in batch_sizes:
            for epochs in epochs_list:
                for units_1 in units_list_1:
                    for units_2 in units_list_2:
                        # Create model with current hyperparameters
                        model = create_model(learning_rate=lr, dropout_rate=dropout_rate, units_1=units_1, units_2=units_2)
                        
                        # Train the model
                        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
                        
                        # Evaluate the model on validation data
                        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
                        
                        # Store the accuracy and parameters
                        params = {'learning_rate': lr, 'dropout_rate': dropout_rate, 'batch_size': batch_size, 'epochs': epochs, 'units_1': units_1, 'units_2': units_2}
                        print("Accuracy:", accuracy, "Params:", params)
                        
                        # Check if current model has the best score
                        if accuracy > best_score:
                            best_score = accuracy
                            best_params = params
                            best_model = model

# Print the best score and parameters
print("Best Mean Test Accuracy:", best_score)
print("Best Parameters:", best_params)

# Evaluate the best model on the test data
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Generate a classification report for the best model
y_pred = best_model.predict_classes(X_test)
report = classification_report(np.argmax(y_test, axis=1), y_pred)
print("Classification Report:")
print(report)


In [ ]:
#experiment_3
# Define a function to create the model
def create_model(learning_rate=0.001, dropout_rate=0.2, units=128):
    model = Sequential()
    model.add(SimpleRNN(units=units, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    model.add(Dropout(dropout_rate))  # Adjust dropout rate
    model.add(BatchNormalization())  # Add BatchNormalization layer
    model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize variables to store best score and parameters
best_score = 0
best_params = {}
best_model = None

# Define lists of hyperparameters to search over
learning_rates = [0.001, 0.0015, 0.002]
dropout_rates = [0.1, 0.2, 0.3, 0.5]
batch_sizes = [32, 64, 128, 256]
epochs_list = [30, 40, 50, 60]
units_list = [64, 128, 256]

# Iterate over all combinations of hyperparameters
for lr in learning_rates:
    for dropout_rate in dropout_rates:
        for batch_size in batch_sizes:
            for epochs in epochs_list:
                for units in units_list:
                    # Create model with current hyperparameters
                    model = create_model(learning_rate=lr, dropout_rate=dropout_rate, units=units)
                    
                    # Train the model
                    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
                    
                    # Evaluate the model on validation data
                    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
                    
                    # Store the accuracy and parameters
                    params = {'learning_rate': lr, 'dropout_rate': dropout_rate, 'batch_size': batch_size, 'epochs': epochs, 'units': units}
                    print("Accuracy:", accuracy, "Params:", params)
                    
                    # Check if current model has the best score
                    if accuracy > best_score:
                        best_score = accuracy
                        best_params = params
                        best_model = model

# Print the best score and parameters
print("Best Mean Test Accuracy:", best_score)
print("Best Parameters:", best_params)

# Evaluate the best model on the test data
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Generate a classification report for the best model
y_pred = best_model.predict_classes(X_test)
report = classification_report(np.argmax(y_test, axis=1), y_pred)
print("Classification Report:")
print(report)

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.1))  # Adjust dropout rate
model.add(BatchNormalization())  # Add BatchNormalization layer
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
    
model.compile(optimizer=Adam(learning_rate=0.0015), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=40, batch_size=128, validation_split=0.2, verbose=1)

In [ ]:
# Reshape test data
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
# Step 1: Obtain predictions on the test data
y_pred_prob = model.predict(X_test_reshaped)
# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Step 2: Generate a classification report
report = classification_report(np.argmax(y_test, axis=1), y_pred)

print("Classification Report:")
print(report)

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model
#model.save('music_genre_classifier_rnn.keras')
#print("Model saved as 'music_genre_classifier_rnn.keras'")